<table align="center">
  <td align="center"><a target="_blank" href="https://colab.research.google.com/github/adiyasan/deepfacetask/face_recognition.ipynb">
        <img src="https://i.ibb.co/2P3SLwK/colab.png"  style="padding-bottom:5px;" />Run in Google Colab</a></td>
  <td align="center"><a target="_blank" href="https://github.com/adiyasan/deepfacetask/face_recognition.ipynb">
        <img src="https://i.ibb.co/xfJbPmL/github.png"  height="70px" style="padding-bottom:5px;"  />View Source on GitHub</a></td>
</table>

# Copyright Information

In [48]:
# install deepface
!pip install deepface

In [49]:
# import all dependencies
from deepface import DeepFace
import matplotlib.pyplot as plt
import os
from os import listdir
import numpy as np
import zipfile

In [ ]:
# # mount google drive
# from google.colab import drive
# drive.mount('/content/drive')

In [ ]:
# load dataset from our github
!wget --no-check-certificate \
    "https://github.com/adiyasan/deepfacetask/archive/refs/heads/master.zip" \
    -O "/content/faces.zip"


zip_ref = zipfile.ZipFile('/content/faces.zip', 'r') #Opens the zip file in read mode
zip_ref.extractall('/content') #Extracts the files into the /content folder
zip_ref.close()

In [52]:
# setting configuration
backends = ['opencv', 'ssd', 'dlib', 'mtcnn', 'retinaface', 'mediapipe']
metrics = ["cosine", "euclidean", "euclidean_l2"]
models = ["VGG-Face", "Facenet", "Facenet512", "OpenFace", "DeepFace", "DeepID", "ArcFace", "Dlib"]
BACKEND = 4
MODEL = 6
METRIC = 0

# the path/directory of dataset
# test_folder_dir = "/content/drive/MyDrive/faces/test"
# ref_folder_dir = "/content/drive/MyDrive/faces/referensi"
test_folder_dir = "/content/deepfacetask-main/images/test"
ref_folder_dir = "/content/deepfacetask-main/images/reference"

In [ ]:
# load test images & find similar images from ref images 
imgs = []
img_paths = []
results = []
valid_images = [".jpg",".jpeg",".png",".jfif"]
for images in os.listdir(test_folder_dir):
  ext = os.path.splitext(images.lower())[1]
  if ext.lower() not in valid_images:
    continue
  print('image = '+images)
  # save img path to array
  img_path = test_folder_dir+"/"+images
  img_paths.append(img_path)
  # save image to array
  img = DeepFace.detectFace(img_path = img_path, detector_backend = backends[BACKEND], 
                            enforce_detection = False, align = True)
  imgs.append(img)
  # face verification
  result = DeepFace.find(img_path = img_path, db_path = ref_folder_dir, model_name = models[MODEL], 
                         detector_backend = backends[BACKEND], distance_metric = metrics[METRIC], 
                         enforce_detection = False, align = True)
  # save result to array
  results.append(result)

# array length
print('image path array length = '+str(len(np.array(img_paths))))
print('image array length = '+str(len(np.array(imgs))))
print('result array length = '+str(len(np.array(results))))
results[0]

In [54]:
# create side by side for image show
side_imgs = []
side_paths = []
for i in range(len(img_paths)):
  side_imgs.append(imgs[i])
  side_paths.append(img_paths[i])
  res_img_path = "null"
  img = "null"
  if len(np.array(results[i]))>0:
    res_img_path = np.array(results[i])[0][0]
    img = DeepFace.detectFace(img_path = res_img_path, 
                              detector_backend = backends[BACKEND], 
                              enforce_detection = False, align = True)
  side_imgs.append(img)
  side_paths.append(res_img_path)

In [55]:
# show side by side test-ref
plt.figure(figsize=(100,100))
for i in range(len(img_paths)*2):
    plt.subplot(len(img_paths),2,i+1)
    plt.xticks([])
    plt.yticks([])
    plt.grid(False)
    if side_imgs[i]=="null":
      plt.imshow(np.zeros((100,100,3), np.uint8))
      plt.xlabel("unknown")
    else:
      plt.imshow(np.array(side_imgs[i]))
      plt.xlabel(side_paths[i])

Output hidden; open in https://colab.research.google.com to view.

In [ ]:
# # show list per figure
# for i in range(len(img_paths)*2):
#   plt.figure(i+1)
#   plt.xticks([])
#   plt.yticks([])
#   if side_imgs[i]=="null":
#     plt.imshow(np.zeros((100,100,3), np.uint8))
#     plt.xlabel("unknown")
#   else:
#     plt.imshow(np.array(side_imgs[i]))
#     plt.xlabel(side_paths[i])

# plt.show()

In [57]:
# setting parameter
THRESHOLD = 0.55

# show distance, label, and result
correctness = 0
for i in range(len(img_paths)):
  test_filename = img_paths[i][len(test_folder_dir)+1:]
  arr_test_filename = test_filename.split('-')

  if len(np.array(results[i]))>0:
    res_img_path = np.array(results[i])[0][0]
    ref_filename = res_img_path[len(ref_folder_dir)+1:]
    arr_ref_filename = ref_filename.split('-')
    distance = np.array(results[i])[0][1]
  else:
    ref_filename = '-'
    distance = 1000
    arr_ref_filename = '-'
  
  if distance>THRESHOLD:
    label = 'unknown'
  else:
    label = arr_ref_filename[0]
  
  if label == arr_test_filename[0]:
    status = True
    correctness +=1
  else:
    status = False
  
  print('\n'+str(i+1)+
        '. '+test_filename+
        ', '+ref_filename+
        ', '+str(distance)+
        ', '+label+
        ', '+str(status))


1. adiyasa-nurfalah4.jpg, riza-ibnu.jpg, 0.6426833653328702, unknown, False

2. riyanto-riyanto2.jpeg, handoko-supeno.jpg, 0.6555746485982019, unknown, False

3. unknown-hasanudin.png, reza-budiawan.jpg, 0.5940616422274076, unknown, True

4. unknown-gozali2.jpeg, imam-ekowicaksono2.jpg, 0.6778899555422833, unknown, True

5. sulthoni-ashiddiiqi2.jpg, sulthoni-ashiddiiqi.jpg, 0.4879417577186591, sulthoni, True

6. unknown-sandiaga-uno.jpg, -, 1000, unknown, True

7. lathifah-arief3.jpeg, lathifah-arief2.jpg, 0.6122549364359583, unknown, False

8. imam-ekowicaksono3.jpg, sartono-arief.jpg, 0.5636188176356336, unknown, False

9. hartanto-tantriawan2.jpg, hartanto-tantriawan.jpg, 0.5022304714463677, hartanto, True

10. unknown-putin.jpeg, -, 1000, unknown, True

11. unknown-revalina.png, -, 1000, unknown, True

12. unknown-claus.jpg, hartanto-tantriawan.jpg, 0.630304783901696, unknown, True

13. sartono-arief2.jpg, riza-ibnu.jpg, 0.5537478445964057, unknown, False

14. handoko-supeno3.jpg,

In [58]:
# print accuracy
accuracy = correctness/len(img_paths)*100
print('No. of test data = '+str(len(img_paths)))
print('No. of correct recognition = '+str(correctness))
print('Accuracy = '+str(accuracy)+' %')

No. of test data = 40
No. of correct recognition = 29
Accuracy = 72.5 %
